# TO-DO's


In [4]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import pandas_ta as ta
import numpy as np
import os
import math
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # Don't wrap columns
pd.set_option('display.max_colwidth', None) # Show full content of each column
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [5]:
Coin = ["BTC-USD", "ETH-USD", "USDT-USD", "BNB-USD", "USDC-USD", "XRP-USD", "DOGE-USD", "ADA-USD","SOL-USD", "DOT-USD", "MATIC-USD", "LTC-USD", "SHIB-USD", "BCH-USD", "LINK-USD", "XLM-USD","AVAX-USD", "ATOM-USD", "HBAR-USD", "FIL-USD", "TRX-USD", "APE-USD", "ICP-USD", "QNT-USD","NEAR-USD", "LDO-USD", "ALGO-USD", "ARB-USD", "EGLD-USD", "VET-USD", "MANA-USD", "FTM-USD","XTZ-USD", "AAVE-USD", "SAND-USD", "STX-USD", "EOS-USD", "CAKE-USD", "KAVA-USD", "CHZ-USD", "THETA-USD", "ENJ-USD", "WAVES-USD", "RUNE-USD", "1INCH-USD", "BAT-USD","OMG-USD", "ZIL-USD", "CRO-USD", "GALA-USD", "YFI-USD", "CRV-USD", "ANKR-USD","SNX-USD", "UNI-USD", "REN-USD", "CELR-USD", "ONT-USD", "RVN-USD", "DASH-USD", "KLAY-USD","FTT-USD", "MINA-USD", "GLMR-USD", "DYDX-USD", "MKR-USD", "ZRX-USD", "BAND-USD","BAL-USD", "ROSE-USD", "FLR-USD", "MOVR-USD", "OP-USD", "TWT-USD", "WOO-USD", "NEXO-USD","HOT-USD", "SUSHI-USD", "SRM-USD", "CVC-USD", "LRC-USD", "QTUM-USD", "ICX-USD", "XEC-USD","RLC-USD", "LSK-USD", "GNO-USD", "STEEM-USD", "WAXP-USD", "SXP-USD", "VTHO-USD", "RENBTC-USD","SPELL-USD", "ILV-USD", "AKT-USD", "PENDLE-USD", "NEO-USD", "AERGO-USD"]
portfolio = ["BTC-USD", "ETH-USD", "USDT-USD", "BNB-USD", "USDC-USD"]

In [16]:
################################################CLASSIC STOCK DATA ANALYSIS###################################################
#bist = endeks
def classic_analysis(bist):
    stock_dict = {}
    stock_data = []
    stock_number = 0
    for stocks in bist:
        try:
            ticker = yf.Ticker(stocks)
            stock_info = ticker.info
            data = {key: stock_info.get(key, None) for key in ["symbol","regularMarketPrice", "priceToBook", "currentPrice", "targetHighPrice", "targetLowPrice", "targetMeanPrice", "targetMedianPrice",
                                                                "bookValue", "open", "dayLow", "dayHigh", "recommendationKey", 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh']}
            historical_data = ticker.history(period="1y",interval="1h")  # Get historical data for the stock
            stock_dict[stocks] = historical_data
            if historical_data.empty:
                print(f"No historical data for {stocks}, skipping...")
                continue  # Skip this ticker if no historical data is found
            stock_number = 1 + stock_number
            print(f"\r{stock_number}/{len(bist)} Downloaded {stocks} data", end='', flush=True)

            historical_data['RSI'] = ta.rsi(historical_data['Close'], length=14)  # Calculate RSI (Relative Strength Index)
            adx_data = ta.adx(historical_data['High'], historical_data['Low'], historical_data['Close'], length=14)  # Calculate ADX (Average Directional Index)
            historical_data['ADX'] = adx_data['ADX_14']
            macd = ta.macd(historical_data['Close'], fast=12, slow=26, signal=9)
            historical_data = pd.concat([historical_data, macd], axis=1)
            # Indicators
            
            historical_data['MACDAS'] = historical_data['MACD_12_26_9'] - historical_data['MACDs_12_26_9']
            historical_data['MACDAS_Signal'] = historical_data['MACDAS'].ewm(span=9, adjust=False).mean()
            historical_data['CCI'] = ta.cci(historical_data['High'], historical_data['Low'], historical_data['Close'], length=20)  # Calculate CCI (Commodity Channel Index)
            historical_data['ROC'] = ta.roc(historical_data['Close'], length=12)  # Calculate ROC (Rate of Change)
            historical_data['ATR'] = ta.atr(historical_data['High'], historical_data['Low'], historical_data['Close'], length=14)  # Calculate ATR (Average True Range)
            historical_data['MFI'] = ta.mfi(historical_data['High'], historical_data['Low'], historical_data['Close'], historical_data['Volume'], length=14)
            historical_data["FWMA"]= ta.fwma(historical_data['Close'],length=14)
            historical_data['OBV'] = ta.obv(historical_data['Close'], historical_data['Volume'])  # Calculate OBV (On-Balance Volume)
            historical_data['CMF'] = ta.cmf(historical_data['High'], historical_data['Low'], historical_data['Close'], historical_data['Volume'])# Calculate CMF (Chaikin Money Flow)
            historical_data['AD'] =  ta.ad(historical_data['High'], historical_data['Low'], historical_data['Close'], historical_data['Volume'])  # Calculate A/D Line (Accumulation/Distribution Line)
            historical_data["VWAP"]= ta.vwap(historical_data['High'], historical_data['Low'], historical_data['Close'], historical_data['Volume'])
            historical_data["kama"]= ta.kama(historical_data['Close'],length=14) # lenght
            supertrend = ta.supertrend(historical_data['High'], historical_data['Low'], historical_data['Close'], length=10, multiplier=3)
            historical_data['Supertrend'] = supertrend['SUPERT_10_3.0']
            historical_data['Supertrend_direction'] = supertrend['SUPERTd_10_3.0']
            
            
            # Calculate Stochastic Oscillator
            stoch = ta.stoch(historical_data['High'], historical_data['Low'], historical_data['Close'], k=14, d=3)
            historical_data['Stoch_K'] = stoch['STOCHk_14_3_3']
            historical_data['Stoch_D'] = stoch['STOCHd_14_3_3']
            
            # Calculate Stochastic RSI
            stochrsi = ta.stochrsi(historical_data['Close'], length=14, rsi_length=14, k=3, d=3)
            historical_data['StochRSI_K'] = stochrsi['STOCHRSIk_14_14_3_3']
            historical_data['StochRSI_D'] = stochrsi['STOCHRSId_14_14_3_3']
 
            # Calculate Bollinger Bands
            bollinger = ta.bbands(historical_data['Close'], length=20, std=2)
            historical_data['BB_Middle'] = bollinger.iloc[:, 1]  # Middle Band
            historical_data['BB_Upper'] = bollinger.iloc[:, 2]  # Upper Band
            historical_data['BB_Lower'] = bollinger.iloc[:, 0]  # Lower Band
            historical_data['BB_BWidth'] = bollinger.iloc[:, 3]  # Bandwidth
            historical_data['BB_%B'] = bollinger.iloc[:, 4]  # %B

            data.update({"RSI": historical_data['RSI'].iloc[-1], "ADX": historical_data['ADX'].iloc[-1],"CCI": historical_data['CCI'].iloc[-1], "ROC": historical_data['ROC'].iloc[-1], "ATR": historical_data['ATR'].iloc[-1], "OBV": historical_data['OBV'].iloc[-1],
                        "CMF": historical_data['CMF'].iloc[-1], "AD": historical_data['AD'].iloc[-1], "MFI": historical_data['MFI'].iloc[-1],
                        "MACD": historical_data['MACD_12_26_9'].iloc[-1],"MACD_signal": historical_data['MACDs_12_26_9'].iloc[-1],"MACD_Hist": historical_data['MACDh_12_26_9'].iloc[-1],"MACDAS": historical_data['MACDAS'].iloc[-1],"MACDAS_Signal": historical_data['MACDAS_Signal'].iloc[-1],
                        "BB_Middle": historical_data['BB_Middle'].iloc[-1],"BB_Upper": historical_data['BB_Upper'].iloc[-1],"BB_Lower": historical_data['BB_Lower'].iloc[-1],'BB_%B': historical_data['BB_%B'].iloc[-1],'BB_BWidth': historical_data['BB_BWidth'].iloc[-1],
                        "FWMA": historical_data["FWMA"].iloc[-1],"VWAP": historical_data["VWAP"].iloc[-1],
                        "kama" : historical_data["kama"].iloc[-1],
                        "Supertrend": historical_data['Supertrend'].iloc[-1],
                        "Supertrend_direction": historical_data['Supertrend_direction'].iloc[-1],
                        "Stoch_K": historical_data['Stoch_K'].iloc[-1],
                        "Stoch_D": historical_data['Stoch_D'].iloc[-1],
                        "StochRSI_K": historical_data['StochRSI_K'].iloc[-1],
                        "StochRSI_D": historical_data['StochRSI_D'].iloc[-1],
                        })
            stock_data.append(data)

        except Exception as e:
            print(f"Error fetching data for {stocks}: {e}")

    # Create a DataFrame from the stock data
    df = pd.DataFrame(stock_data)
    df["currentPrice"] = df["regularMarketPrice"]
    df["YF%"] = ((df["currentPrice"] - df["fiftyTwoWeekLow"])/(df["fiftyTwoWeekHigh"]-df["fiftyTwoWeekLow"]))*100 #Check the differences
    df["MACDAS-dif"] = df["MACDAS"] - df["MACDAS_Signal"] 
    df["change"] = ((df["regularMarketPrice"] / df["open"]) - 1) * 100
    #df["change"] = ((df["currentPrice"] / df["open"]) - 1) * 100
    df["BB_Pot"] = ((df['BB_Upper'] / df["currentPrice"]) - 1) * 100
    df["BB_Opt"] = ((df['BB_Lower'] / df["currentPrice"]) - 1) * 100
    df["TrendWay"] = np.select([(df["ADX"] > 20) & (df["ROC"] > 0), (df["ADX"] > 20) & (df["ROC"] <= 0), (df["ADX"] <= 20)], ["upper", "lower", "no-trend"], default="unknown")

    df1 = df[["symbol", "Stoch_K","Stoch_D","StochRSI_K","StochRSI_D","MACDAS-dif","MACDAS","MACDAS_Signal", 'BB_BWidth', 'BB_%B',"YF%","MFI", "OBV", "RSI","AD", "ADX", "CCI", "ROC","CMF", "ATR","FWMA","VWAP","Supertrend","Supertrend_direction","kama", "currentPrice","change"]]
    df2 = df1.rename(columns={"currentPrice": "CPrice","Supertrend_direction":"st_way"})

    pd.set_option('display.float_format', '{:.2f}'.format)
    print("\nDownload process is done!")
    df2
    now = datetime.now()  # This part will copy for our sell lists. Especially further analysis.
    formatted_time = now.strftime("%m-%d_%H-%M-%S")
    #df2.to_csv(f"CSVs/Results/{formatted_time}.csv")
    df2.to_csv(f"CSVs/Coin_last.csv")
    print(df2)
#classic_analysis(portfolio)

In [17]:
classic_analysis(portfolio) 

Error fetching data for BTC-USD: Failed to perform, curl: (16) . See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
4/5 Downloaded USDC-USD data
Download process is done!
     symbol  Stoch_K  Stoch_D  StochRSI_K  StochRSI_D  MACDAS-dif  MACDAS  \
0   ETH-USD    21.93    17.39       33.40       19.51       -0.49   -1.13   
1  USDT-USD    70.11    58.96       52.96       30.23        0.00    0.00   
2   BNB-USD     9.88     4.89       19.85        9.77       -0.29   -0.55   
3  USDC-USD    55.88    46.42       66.89       44.31        0.00    0.00   

   MACDAS_Signal  BB_BWidth  BB_%B   YF%   MFI             OBV   RSI  \
0          -0.64       2.59   0.22 40.93 86.43 134045493760.00 38.97   
1          -0.00       0.04   0.69 57.61 80.83 -80479684608.00 54.05   
2          -0.26       2.07   0.14 62.99 45.73  24854369920.00 34.43   
3          -0.00       0.03   1.04 40.43 18.95  81281116288.00 57.93   

                 AD   ADX     CCI   ROC   CMF   ATR    FWMA

In [13]:
df2 = pd.read_csv(f"CSVs/Coin_last.csv")
df2

,Unnamed: 0,symbol,Stoch_K,Stoch_D,StochRSI_K,StochRSI_D,MACDAS-dif,MACDAS,MACDAS_Signal,BB_BWidth,BB_%B,YF%,MFI,OBV,RSI,AD,ADX,CCI,ROC,CMF,ATR,FWMA,VWAP,Supertrend,st_way,kama,CPrice,change
0,0,BTC-USD,24.37,17.60,34.20,21.42,-24.48,-32.60,-8.12,1.05,0.33,87.68,92.83,313471272960.00,47.61,236219145443.00,18.57,-74.31,-0.22,0.57,309.74,104146.91,104221.83,103843.68,1,104374.80,104225.20,-0.40
1,1,ETH-USD,22.23,17.49,34.35,19.82,-0.46,-1.10,-0.63,2.59,0.23,40.95,86.39,134025625088.00,39.32,168409491987.33,32.25,-92.92,-1.00,0.10,14.09,2499.31,2501.60,2535.46,-1,2515.59,2500.66,-1.13
2,2,USDT-USD,56.06,54.28,31.99,23.24,-0.00,-0.00,-0.00,0.04,0.10,53.79,80.83,-80479684608.00,45.10,-1123426323280.73,19.96,-117.28,-0.01,-0.21,0.00,1.00,1.00,1.00,-1,1.00,1.00,-0.02
3,3,BNB-USD,10.01,4.93,20.12,9.86,-0.29,-0.55,-0.26,2.07,0.14,63.00,45.73,24854369920.00,34.60,19363903981.50,30.70,-129.82,-0.94,-0.00,2.03,650.75,650.89,656.05,-1,652.99,650.59,-1.18
4,4,USDC-USD,32.54,38.64,33.55,33.19,-0.00,-0.00,-0.00,0.03,-0.14,31.12,14.59,81166130304.00,41.35,-117802868328.89,15.11,-267.57,-0.02,0.43,0.00,1.00,1.00,1.00,1,1.00,1.00,-0.03


In [18]:
cheap = df2.copy()                         #we can catch early bullish stocks with this sample
# CMF  C. Money Flow index important
# ATR  Average True Range but I don't know the real value	
# FWMA	Fibonacci
# VWAP	Volume-weighted average price I don't know 
cheap = cheap[cheap["CCI"] < -80]
#cheap = cheap[cheap["MFI"] < 30] # Money Flow Index
cheap = cheap[cheap["RSI"] < 40]
cheap = cheap[cheap["YF%"] < 40]
cheap = cheap[cheap["ADX"] < 25]
cheap = cheap[cheap["BB_%B"] < 0.4] # We can miss the upward movements 
#cheap = cheap[cheap["ROC"] > 0] # Momentum tool (rate of change) 

#cheap = cheap[cheap["MACDAS"] > cheap["MACDAS_Signal"]]
#cheap = cheap[cheap["Stoch_K"] < cheap["Stoch_D"]]
cheap.head(50)
cheap["time"] = datetime.now()
cheap = cheap[["symbol",'BB_BWidth', 'BB_%B',"YF%","MFI","RSI","ADX", "CCI", "ROC","CMF", "ATR","FWMA","VWAP","st_way","Supertrend","kama","CPrice","change","time"]]
cheap = cheap.sort_values(by=["time","RSI"])
#cheap.to_csv(f"CSVs/Cheap.csv")

file_path = "CSVs/CheapCoin.csv"

if os.path.exists(file_path):
    old_data = pd.read_csv(file_path)
    combined = pd.concat([old_data, cheap], ignore_index=True)
else:
    combined = cheap

# Save the updated file
combined.to_csv(file_path, index=False)
cheap

,symbol,BB_BWidth,BB_%B,YF%,MFI,RSI,ADX,CCI,ROC,CMF,ATR,FWMA,VWAP,st_way,Supertrend,kama,CPrice,change,time
85,LSK-USD,31.87,-0.01,3.19,22.36,30.44,17.60,-184.18,-18.42,-0.11,0.03,0.45,0.43,-1,0.51,0.49,0.43,-1.53,2025-06-01 15:03:02.853414
52,ANKR-USD,31.27,0.05,4.37,31.95,32.53,20.34,-170.17,-16.22,-0.11,0.00,0.02,0.02,-1,0.02,0.02,0.02,-1.07,2025-06-01 15:03:02.853414
91,RENBTC-USD,15.00,-0.07,2.39,38.30,32.95,21.22,-204.46,-10.41,-0.28,1044.47,15287.28,15284.32,-1,18004.30,16170.54,14894.14,-0.91,2025-06-01 15:03:02.853414
84,RLC-USD,32.37,0.02,1.69,25.60,33.26,14.53,-185.77,-16.98,-0.31,0.08,1.05,1.01,-1,1.26,1.15,1.00,-2.95,2025-06-01 15:03:02.853414
62,MINA-USD,32.98,-0.00,2.80,29.12,33.46,16.01,-211.29,-17.69,-0.12,0.02,0.22,0.21,-1,0.26,0.24,0.21,-1.78,2025-06-01 15:03:02.853414
12,SHIB-USD,24.37,0.05,12.20,30.39,33.86,21.37,-197.18,-14.62,-0.26,0.00,0.00,0.00,-1,0.00,0.00,0.00,-0.11,2025-06-01 15:03:02.853414
89,SXP-USD,29.56,0.07,8.72,34.45,34.10,16.27,-164.75,-13.65,-0.07,0.01,0.18,0.17,-1,0.21,0.19,0.17,-0.37,2025-06-01 15:03:02.853414
57,ONT-USD,29.98,0.08,6.55,35.53,34.24,19.56,-153.14,-15.31,-0.05,0.01,0.13,0.13,-1,0.15,0.14,0.13,-0.43,2025-06-01 15:03:02.853414
80,LRC-USD,39.73,0.07,4.08,24.01,34.39,22.44,-148.77,-21.71,-0.21,0.01,0.09,0.09,-1,0.11,0.10,0.09,-0.20,2025-06-01 15:03:02.853414
18,HBAR-USD,26.45,0.03,34.81,36.44,34.43,17.86,-172.97,-14.60,-0.10,0.01,0.17,0.17,-1,0.20,0.18,0.17,-0.62,2025-06-01 15:03:02.853414


In [20]:
daily_trades = df2.copy()  # Create a copy of the dataframe for daily trading signals

# Filter for daily trading signals - focusing on momentum and trend indicators
# ADX > 20 indicates a stronger trend
# ROC > 0 indicates positive momentum
# Check if price is above VWAP (bullish signal for day trading)
# Check if price is crossing above key moving average (kama)

daily_trades = daily_trades[daily_trades["ADX"] > 20]  # Strong trend
daily_trades = daily_trades[daily_trades["ROC"] > 0]   # Positive momentum
#daily_trades = daily_trades[daily_trades["CPrice"] > daily_trades["VWAP"]]  # Price above VWAP
daily_trades = daily_trades[daily_trades["st_way"] > 0]  # Supertrend indicator is positive

# Add a volatility filter - we want enough movement for day trading
daily_trades = daily_trades[daily_trades["BB_BWidth"] > 1.5]  # Bollinger Band width indicating volatility

# Look for recent bullish reversal signals
daily_trades = daily_trades[daily_trades["RSI"] > 40]  # Not oversold, starting to move up
daily_trades = daily_trades[daily_trades["RSI"] < 70]  # Not overbought yet

# Sort results by momentum and trend strength
daily_trades["score"] = (daily_trades["ROC"] * 0.4) + \
                         (daily_trades["ADX"] * 0.3) + \
                         ((daily_trades["RSI"] - 40) * 0.3)  # Custom scoring system

# Mark timestamp
daily_trades["time"] = datetime.now()

# Select relevant columns and sort
daily_trades = daily_trades[["symbol", 'BB_BWidth', 'BB_%B', "YF%", "MFI", "RSI", "ADX", 
                             "CCI", "ROC", "CMF", "ATR", "FWMA", "VWAP", "st_way", 
                             "Supertrend", "kama", "CPrice", "change", "score", "time"]]
daily_trades = daily_trades.sort_values(by=["score"], ascending=False)

## Create a folder if it doesn't exist
#os.makedirs("CSVs", exist_ok=True)
#
# Save data to CSV with timestamp to track daily signals
#timestamp = datetime.now().strftime("%Y%m%d")
#file_path = f"CSVs/DailyTrades_{timestamp}.csv"
#
## Update or create the CSV file
#if os.path.exists(file_path):
#    old_data = pd.read_csv(file_path)
#    combined = pd.concat([old_data, daily_trades], ignore_index=True)
#    # Remove duplicates based on symbol (keep the most recent)
#    combined = combined.sort_values('time').drop_duplicates(subset=['symbol'], keep='last')
#else:
#    combined = daily_trades
#
## Save the updated file
#combined.to_csv(file_path, index=False)
#
## Also maintain a historical log of all signals
#history_file = "CSVs/DailyTradesHistory.csv"
#if os.path.exists(history_file):
#    history_data = pd.read_csv(history_file)
#    history_combined = pd.concat([history_data, daily_trades], ignore_index=True)
#else:
#    history_combined = daily_trades
#
#history_combined.to_csv(history_file, index=False)
#
## Print top 10 trading candidates
#print(f"Top daily trading candidates for {timestamp}:")
#display(daily_trades.head(10))
daily_trades.head(20)

,symbol,BB_BWidth,BB_%B,YF%,MFI,RSI,ADX,CCI,ROC,CMF,ATR,FWMA,VWAP,st_way,Supertrend,kama,CPrice,change,score,time
23,QNT-USD,29.52,0.80,49.02,61.04,61.74,36.76,90.06,13.18,0.01,6.48,108.82,109.23,1,95.57,99.07,109.62,2.01,22.83,2025-06-01 15:07:00.230859
82,ICX-USD,19.81,0.56,19.93,69.42,52.20,34.61,47.88,7.02,-0.01,0.01,0.12,0.12,1,0.10,0.11,0.12,-4.13,16.85,2025-06-01 15:07:00.230859
3,BNB-USD,8.51,0.29,63.00,46.63,48.87,30.05,-55.18,0.04,0.10,17.86,659.80,652.66,1,629.06,651.42,650.61,-1.18,11.69,2025-06-01 15:07:00.230859
13,BCH-USD,12.71,0.36,38.96,46.50,50.16,23.36,-26.45,2.09,-0.02,20.49,407.70,404.97,1,372.39,383.59,402.32,-2.95,10.89,2025-06-01 15:07:00.230859


In [ ]:
# we can use this function for similar metrics like rsi and mfi 
condition = (df2['CCI'] < -100) | (df2['RSI'] < 30) | (df2['YF%'] < -20) | (df2['MFI'] < 20)
filtered_df = df2[condition].copy()
filtered_df

,Unnamed: 0,symbol,Stoch_K,Stoch_D,StochRSI_K,StochRSI_D,MACDAS-dif,MACDAS,MACDAS_Signal,BB_BWidth,BB_%B,YF%,MFI,OBV,RSI,AD,ADX,CCI,ROC,CMF,ATR,FWMA,VWAP,Supertrend,st_way,kama,CPrice,change
5,5,XRP-USD,12.72,9.46,7.73,4.34,-0.01,-0.03,-0.02,19.27,0.06,58.52,23.99,242741279262.00,39.25,191753688662.71,17.51,-162.93,-8.98,-0.08,0.10,2.18,2.15,2.44,-1,2.27,2.15,-1.38
6,6,DOGE-USD,5.59,7.62,0.00,0.00,-0.00,-0.01,-0.00,26.30,-0.05,26.97,34.25,108851587632.00,38.67,41300575831.98,25.16,-229.38,-16.22,-0.06,0.01,0.20,0.19,0.23,-1,0.21,0.19,-1.63
7,7,ADA-USD,6.97,9.19,0.00,0.00,-0.01,-0.01,-0.01,20.50,-0.06,36.99,31.54,40353880446.00,36.30,7402205796.26,19.43,-217.60,-10.94,-0.10,0.04,0.69,0.67,0.79,-1,0.73,0.66,-3.19
8,8,SOL-USD,5.40,13.91,0.59,0.40,-1.64,-3.57,-1.93,18.60,-0.07,28.11,51.82,-42377350633.00,38.78,17706045398.58,26.62,-209.94,-9.68,-0.10,8.45,158.35,153.23,150.95,1,165.94,152.18,-2.83
9,9,DOT-USD,8.32,7.72,1.62,3.79,-0.03,-0.10,-0.07,25.75,0.03,9.25,42.42,3430096587.00,38.03,3190485869.92,17.22,-175.29,-13.82,-0.17,0.26,4.15,4.04,4.80,-1,4.46,4.03,-1.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,92,SPELL-USD,13.56,9.74,13.57,6.99,-0.00,-0.00,-0.00,28.27,0.03,8.00,51.14,1304350770.00,38.45,-1316696607.40,19.71,-191.91,-14.51,-0.14,0.00,0.00,0.00,0.00,-1,0.00,0.00,-1.25
93,93,ILV-USD,13.24,16.73,13.33,20.23,-0.02,-0.18,-0.16,27.94,0.20,3.13,47.35,-6131903.00,41.71,301727523.07,14.58,-125.20,-6.17,-0.13,1.05,13.02,12.80,15.74,-1,14.65,12.82,0.29
94,94,AKT-USD,8.45,5.73,6.66,3.49,-0.01,-0.05,-0.04,35.50,0.04,10.07,38.01,1618588908.00,38.05,42343245.11,28.92,-188.02,-16.11,-0.20,0.13,1.32,1.28,1.63,-1,1.51,1.27,-1.00
96,96,NEO-USD,11.71,16.38,4.45,4.00,-0.05,-0.12,-0.07,23.26,0.01,6.76,39.42,398557867.00,39.27,219917484.97,19.74,-203.07,-11.67,-0.09,0.40,5.99,5.81,6.98,-1,6.31,5.80,-1.60


In [9]:
# Momentum stocks filter
momentum = df2.copy()
momentum = momentum[(momentum["RSI"] >=50) &(momentum["RSI"]< 70)]  # Strong momentum
#momentum = momentum[momentum["MACDAS"] > momentum["MACDAS_Signal"]]  # MACD above signal line
momentum = momentum[momentum["ROC"] > 0]  # Positive rate of change
momentum = momentum[momentum["ADX"] > 20]  # Strong trend
momentum = momentum[momentum["CCI"] < 100]  # Strong trend
#momentum = momentum[momentum["BB_%B"] > 0.5]
momentum = momentum[momentum["kama"] < momentum["CPrice"]]
momentum = momentum[momentum["Supertrend"] < momentum["CPrice"]]  # MACD above signal line
momentum = momentum[momentum["Stoch_K"] < momentum["Stoch_D"]]
momentum = momentum.sort_values(by="ROC", ascending=False)  # Sort by strongest momentum
momentum = momentum[["symbol",'BB_BWidth', 'BB_%B',"YF%","MFI","RSI","ADX", "CCI", "ROC","CMF", "ATR","FWMA","VWAP","st_way","Supertrend","kama","CPrice","change"]]
momentum.head(50)

,symbol,BB_BWidth,BB_%B,YF%,MFI,RSI,ADX,CCI,ROC,CMF,ATR,FWMA,VWAP,st_way,Supertrend,kama,CPrice,change
23,QNT-USD,29.52,0.80,49.02,61.04,61.74,36.76,90.06,13.18,0.01,6.48,108.82,109.23,1,95.57,99.07,109.62,2.01
82,ICX-USD,19.81,0.56,19.93,69.42,52.20,34.61,47.88,7.02,-0.01,0.01,0.12,0.12,1,0.10,0.11,0.12,-4.13
13,BCH-USD,12.71,0.36,38.96,46.50,50.16,23.36,-26.45,2.09,-0.02,20.49,407.70,404.97,1,372.39,383.59,402.32,-2.95


In [10]:
top_performers = df2.copy()
top_performers = top_performers[top_performers["change"]>= 6]
top_performers

,Unnamed: 0,symbol,Stoch_K,Stoch_D,StochRSI_K,StochRSI_D,MACDAS-dif,MACDAS,MACDAS_Signal,BB_BWidth,BB_%B,YF%,MFI,OBV,RSI,AD,ADX,CCI,ROC,CMF,ATR,FWMA,VWAP,Supertrend,st_way,kama,CPrice,change


In [11]:
#If this one works write portfolio as same format.
XU100 = df2.copy()
XU100 = XU100[XU100["symbol"].isin(bist100)]
XU100 = XU100.sort_values(by="RSI", ascending=True)
XU100.head(50)
XU100.to_csv(f"CSVs/XU100 last.csv")

In [12]:
#gemini üzerinden seçtiğimiz hisselerin haberlerini getirecek bir kod yazabiliriz.
# Change'e göre sıralayıp listeyi inceleyelim
hisse = "EGPRO.IS"
solo_stock = df2.copy()
solo_stock = solo_stock[solo_stock["symbol"]==hisse]
#print(solo_stock)
solo_stock

,Unnamed: 0,symbol,Stoch_K,Stoch_D,StochRSI_K,StochRSI_D,MACDAS-dif,MACDAS,MACDAS_Signal,BB_BWidth,BB_%B,YF%,MFI,OBV,RSI,AD,ADX,CCI,ROC,CMF,ATR,FWMA,VWAP,Supertrend,st_way,kama,CPrice,change


In [ ]:
def find_value_opportunities(df, save_history=True):
    """
    Identify potentially undervalued stocks using technical indicators and multi-factor approach.
    
    Parameters:
    df - DataFrame with technical indicators
    save_history - Whether to save results to historical file for tracking
    
    Returns:
    DataFrame of filtered value opportunities
    """
    # Create deep copy to avoid modifying original data
    value_opps = df.copy()
    
    # Create composite technical score using multiple factors
    # This helps prevent false signals from any single indicator
    value_opps['oversold_score'] = 0
    
    # Add points to score based on oversold conditions
    value_opps.loc[value_opps["CCI"] < -100, 'oversold_score'] += 2  # Deeply oversold
    value_opps.loc[(value_opps["CCI"] >= -100) & (value_opps["CCI"] < -80), 'oversold_score'] += 1  # Moderately oversold
    
    value_opps.loc[value_opps["RSI"] < 30, 'oversold_score'] += 2  # Deeply oversold
    value_opps.loc[(value_opps["RSI"] >= 30) & (value_opps["RSI"] < 40), 'oversold_score'] += 1  # Moderately oversold
    
    value_opps.loc[value_opps["MFI"] < 20, 'oversold_score'] += 2  # Deeply oversold
    value_opps.loc[(value_opps["MFI"] >= 20) & (value_opps["MFI"] < 30), 'oversold_score'] += 1  # Moderately oversold
    
    # Bollinger Band related signals
    value_opps.loc[value_opps["BB_%B"] < 0.2, 'oversold_score'] += 1  # Price near lower band
    
    # Filter for stocks with meaningful oversold scores
    value_opps = value_opps[value_opps['oversold_score'] >= 3]
    
    # Check for reversal signals that indicate potential bottoming
    value_opps['reversal_signals'] = 0
    
    # MACD crossover signal (early momentum change)
    value_opps.loc[value_opps["MACDAS"] > value_opps["MACDAS_Signal"], 'reversal_signals'] += 1
    
    # Stochastic reversal (momentum change)
    if "Stoch_K" in value_opps.columns and "Stoch_D" in value_opps.columns:
        value_opps.loc[value_opps["Stoch_K"] > value_opps["Stoch_D"], 'reversal_signals'] += 1
    
    # Volume confirmation (accumulation indicates smart money buying)
    value_opps.loc[value_opps["CMF"] > 0, 'reversal_signals'] += 1  # Positive money flow
    
    # Give preference to stocks showing signs of reversal
    value_opps = value_opps.sort_values(by=["oversold_score", "reversal_signals", "RSI"], ascending=[False, False, True])
    
    # Add timing information
    value_opps["scan_time"] = datetime.now()
    
    # Select relevant columns for output
    result = value_opps[["symbol", "oversold_score", "reversal_signals", 'BB_BWidth', 'BB_%B', 
                        "YF%", "MFI", "RSI", "ADX", "CCI", "ROC", "CMF", "ATR",
                        "FWMA", "VWAP", "st_way", "Supertrend", "kama", "CPrice", 
                        "change", "scan_time"]]
    
    # Save historical data for tracking performance over time
    if save_history:
        file_path = "CSVs/ValueOpportunities.csv"
        
        if os.path.exists(file_path):
            try:
                old_data = pd.read_csv(file_path)
                
                # Keep only the most recent 90 days of data to prevent file bloat
                if 'scan_time' in old_data.columns:
                    old_data['scan_time'] = pd.to_datetime(old_data['scan_time'])
                    cutoff_date = datetime.now() - timedelta(days=90)
                    old_data = old_data[old_data['scan_time'] > cutoff_date]
                
                combined = pd.concat([old_data, result], ignore_index=True)
            except Exception as e:
                print(f"Error reading history file: {e}")
                combined = result
        else:
            combined = result
            
        # Save the updated file
        combined.to_csv(file_path, index=False)
    
    return result
find_value_opportunities(df2)

Error reading history file: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 76 and the array at index 1 has size 74


,symbol,oversold_score,reversal_signals,BB_BWidth,BB_%B,YF%,MFI,RSI,ADX,CCI,ROC,CMF,ATR,FWMA,VWAP,st_way,Supertrend,kama,CPrice,change,scan_time
30,MANA-USD,6,1,34.30,0.05,11.65,15.21,35.37,19.81,-161.20,-16.50,-0.16,0.02,0.28,0.27,-1,0.33,0.31,0.27,-1.60,2025-06-01 15:00:26.351350
84,RLC-USD,5,1,32.37,0.02,1.69,25.60,33.26,14.53,-185.77,-16.98,-0.31,0.08,1.05,1.01,-1,1.26,1.15,1.00,-2.95,2025-06-01 15:00:26.351350
80,LRC-USD,5,1,39.73,0.07,4.08,24.01,34.39,22.44,-148.77,-21.71,-0.21,0.01,0.09,0.09,-1,0.11,0.10,0.09,-0.20,2025-06-01 15:00:26.351350
45,BAT-USD,5,1,27.99,0.04,7.54,28.05,35.79,16.87,-168.04,-14.95,-0.17,0.01,0.13,0.13,-1,0.15,0.14,0.13,-1.59,2025-06-01 15:00:26.351350
5,XRP-USD,5,1,19.27,0.06,58.52,23.99,39.25,17.51,-162.93,-8.98,-0.08,0.10,2.18,2.15,-1,2.44,2.27,2.15,-1.38,2025-06-01 15:00:26.351350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,DASH-USD,3,0,16.13,0.09,7.45,50.81,42.17,12.75,-180.43,-6.66,-0.06,1.11,22.32,21.92,1,21.41,23.00,22.00,0.48,2025-06-01 15:00:26.351350
22,ICP-USD,3,0,19.55,0.10,5.84,49.42,42.26,15.07,-158.68,-7.25,-0.06,0.31,4.97,4.88,-1,5.78,5.25,4.88,-0.63,2025-06-01 15:00:26.351350
65,MKR-USD,3,0,22.13,0.13,31.89,48.99,42.75,14.91,-132.02,-9.41,-0.14,95.73,1588.49,1564.11,1,1543.63,1642.72,1563.87,-0.51,2025-06-01 15:00:26.351350
79,CVC-USD,3,0,28.20,0.13,28.07,41.03,43.64,31.54,-111.69,-10.83,-0.11,0.01,0.13,0.12,1,0.12,0.13,0.12,-1.69,2025-06-01 15:00:26.351350


In [ ]:
def backtest_value_strategy(stock_symbols, lookback_period='1y', holding_period=30):
    """
    Backtest the value opportunity strategy
    
    Parameters:
    stock_symbols - List of stock symbols to test
    lookback_period - Period of historical data to use
    holding_period - How many days to hold stocks after signal
    
    Returns:
    DataFrame with backtest results
    """
    results = []
    
    for symbol in stock_symbols:
        try:
            # Get historical data
            ticker = yf.Ticker(symbol)
            hist_data = ticker.history(period=lookback_period)
            
            if hist_data.empty:
                continue
                
            # Calculate all technical indicators as in your classic_analysis function
            hist_data['RSI'] = ta.rsi(hist_data['Close'], length=14)
            # Add all other indicators from your original code...
            
            # Get signals using your value criteria
            signals = pd.DataFrame(index=hist_data.index)
            signals['symbol'] = symbol
            signals['price'] = hist_data['Close']
            
            # Define entry condition (equivalent to the value_opportunities function)
            signals['oversold'] = ((hist_data['CCI'] < -80) & 
                                  (hist_data['RSI'] < 35))
            
            # Find signal dates
            signal_dates = signals[signals['oversold']].index
            
            # Calculate returns for holding period after each signal
            for date in signal_dates:
                try:
                    entry_price = hist_data.loc[date, 'Close']
                    exit_idx = min(hist_data.index.get_loc(date) + holding_period, len(hist_data) - 1)
                    exit_date = hist_data.index[exit_idx]
                    exit_price = hist_data.loc[exit_date, 'Close']
                    
                    pct_return = (exit_price / entry_price - 1) * 100
                    
                    results.append({
                        'symbol': symbol,
                        'entry_date': date,
                        'exit_date': exit_date,
                        'entry_price': entry_price,
                        'exit_price': exit_price,
                        'return': pct_return
                    })
                except Exception as e:
                    print(f"Error calculating returns for {symbol} on {date}: {e}")
                    
        except Exception as e:
            print(f"Error processing {symbol}: {e}")
            
    return pd.DataFrame(results)
backtest_value_strategy(portfolio)

Error processing KRDMD.IS: 'CCI'
Error processing BRSAN.IS: 'CCI'
Error processing ARCLK.IS: 'CCI'
Error processing HEKTS.IS: 'CCI'
Error processing MIATK.IS: 'CCI'
Error processing ZOREN.IS: 'CCI'
Error processing SMRTG.IS: 'CCI'
Error processing MAVI.IS: 'CCI'
Error processing TTRAK.IS: 'CCI'
Error processing EGPRO.IS: 'CCI'
Error processing AEFES.IS: 'CCI'
Error processing FROTO.IS: 'CCI'
Error processing ALCAR.IS: 'CCI'


""


In [15]:
len("Alternative Data | US | 300000 Unique Company | AI Enhanced & Human Validated, Double checked from Indeed & Google Maps")

119